# **LP1_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [4]:
problem = """A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, an 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm. """

In [5]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Let the variables be defined as follows:

Let's denote the litres of C1 used in the products GCA, GCB and GCC GCA_C1, GCB_C1, and GCC_C1 respectively.
Let's denote the litres of C2 used in the products GCA, GCB, and GCC GCA_C2, GCB_C2, and GCC_C2 respectively.

x1 = Volume of GCA produced (in litres)
x2 = Volume of GCB produced (in litres)
x3 = Volume of GCC produced (in litres)

Critical component variables:
CC_GCA = Volume of critical component in GCA (in litres)
CC_GCB = Volume of critical component in GCB (in litres)
CC_GCC = Volume of critical component in GCC (in litres)


In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
print(response2.choices[0].message.content)

The objective function to maximize the profit from the sale of each litre of GCA, GCB, and GCC is:

Maximize P = 120*x1 + 135*x2 + 155*x3


In [9]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [10]:
print(response3.choices[0].message.content)

Since you are only asking for the constraints, these would be:

1. For the supply of chemical components:
- GCA_C1 + GCB_C1 + GCC_C1 ≤ 10,000
- GCA_C2 + GCB_C2 + GCC_C2 ≤ 15,000

2. For the demand of fluids:
- x1 ≥ 6,000
- x2 ≥ 7,000
- x3 ≥ 9,000

3. For the mixing without loss or gain in volume:
- GCA_C1 + GCA_C2 = x1
- GCB_C1 + GCB_C2 = x2
- GCC_C1 + GCC_C2 = x3

4. For the content of the critical element:
- CC_GCA = 0.4*GCA_C1 + 0.2*GCA_C2 ≥ 0.3*x1
- CC_GCB = 0.4*GCB_C1 + 0.2*GCB_C2 ≤ 0.3*x2
- 0.4*GCC_C1 ≥ 0.3*(GCC_C1 + GCC_C2) 

5. All variables must be non-negative:
- GCA_C1, GCB_C1, GCC_C1, GCA_C2, GCB_C2, GCC_C2, x1, x2, x3 ≥ 0


### **Generate Pyomo Code**

In [11]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [12]:
Markdown(response4.choices[0].message.content)

Here is a sample Pyomo code according to the provided constraints and the objective function:

```python
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)
model.x3 = Var(within=NonNegativeReals)
model.GCA_C1 = Var(within=NonNegativeReals)
model.GCA_C2 = Var(within=NonNegativeReals)
model.GCB_C1 = Var(within=NonNegativeReals)
model.GCB_C2 = Var(within=NonNegativeReals)
model.GCC_C1 = Var(within=NonNegativeReals)
model.GCC_C2 = Var(within=NonNegativeReals)

# Objective function
model.profit = Objective(expr = 120*model.x1 + 135*model.x2 + 155*model.x3, sense=maximize)

# Constraints
model.total_component_C1 = Constraint(expr = model.GCA_C1 + model.GCB_C1 + model.GCC_C1 <= 10000)
model.total_component_C2 = Constraint(expr = model.GCA_C2 + model.GCB_C2 + model.GCC_C2 <= 15000)
model.minimum_GCA = Constraint(expr = model.x1 >= 6000)
model.minimum_GCB = Constraint(expr = model.x2 >= 7000)
model.minimum_GCC = Constraint(expr = model.x3 >= 9000)

model.mixing_volume_GCA = Constraint(expr = model.GCA_C1 + model.GCA_C2 == model.x1)
model.mixing_volume_GCB = Constraint(expr = model.GCB_C1 + model.GCB_C2 == model.x2)
model.mixing_volume_GCC = Constraint(expr = model.GCC_C1 + model.GCC_C2 == model.x3)

model.critical_component_GCA = Constraint(expr = 0.4*model.GCA_C1 + 0.2*model.GCA_C2 >= 0.3*model.x1)
model.critical_component_GCB = Constraint(expr = 0.4*model.GCB_C1 + 0.2*model.GCB_C2 <= 0.3*model.x2)
model.critical_component_GCC = Constraint(expr = 0.4*model.GCC_C1 >= 0.3*(model.GCC_C1 + model.GCC_C2))

# Print the model
model.pprint()

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print('Profit = ', model.profit())
print('Litres of GCA = ', model.x1())
print('Litres of GCB = ', model.x2())
print('Litres of GCC = ', model.x3())
```

Please note that solving the model will return the optimal point corresponding to the maximal objective function value that still satisfies all constraints.

### **Run the code Generated by GPT4**

In [13]:
from pyomo.environ import *

# Create a model
model = ConcreteModel()

# Variables
model.x1 = Var(within=NonNegativeReals)
model.x2 = Var(within=NonNegativeReals)
model.x3 = Var(within=NonNegativeReals)
model.GCA_C1 = Var(within=NonNegativeReals)
model.GCA_C2 = Var(within=NonNegativeReals)
model.GCB_C1 = Var(within=NonNegativeReals)
model.GCB_C2 = Var(within=NonNegativeReals)
model.GCC_C1 = Var(within=NonNegativeReals)
model.GCC_C2 = Var(within=NonNegativeReals)

# Objective function
model.profit = Objective(expr = 120*model.x1 + 135*model.x2 + 155*model.x3, sense=maximize)

# Constraints
model.total_component_C1 = Constraint(expr = model.GCA_C1 + model.GCB_C1 + model.GCC_C1 <= 10000)
model.total_component_C2 = Constraint(expr = model.GCA_C2 + model.GCB_C2 + model.GCC_C2 <= 15000)
model.minimum_GCA = Constraint(expr = model.x1 >= 6000)
model.minimum_GCB = Constraint(expr = model.x2 >= 7000)
model.minimum_GCC = Constraint(expr = model.x3 >= 9000)

model.mixing_volume_GCA = Constraint(expr = model.GCA_C1 + model.GCA_C2 == model.x1)
model.mixing_volume_GCB = Constraint(expr = model.GCB_C1 + model.GCB_C2 == model.x2)
model.mixing_volume_GCC = Constraint(expr = model.GCC_C1 + model.GCC_C2 == model.x3)

model.critical_component_GCA = Constraint(expr = 0.4*model.GCA_C1 + 0.2*model.GCA_C2 >= 0.3*model.x1)
model.critical_component_GCB = Constraint(expr = 0.4*model.GCB_C1 + 0.2*model.GCB_C2 <= 0.3*model.x2)
model.critical_component_GCC = Constraint(expr = 0.4*model.GCC_C1 >= 0.3*(model.GCC_C1 + model.GCC_C2))

# Print the model
model.pprint()

# Solve the model
solver = SolverFactory('glpk')
solver.solve(model)

# Print the results
print('Profit = ', model.profit())
print('Litres of GCA = ', model.x1())
print('Litres of GCB = ', model.x2())
print('Litres of GCC = ', model.x3())

9 Var Declarations
    GCA_C1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    GCA_C2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    GCB_C1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    GCB_C2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    GCC_C1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : NonNegativeReals
    GCC_C2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :  None :  None : False :  True : No

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**